# Практическая работа «Маскирование слов»

Используя модель BERT и её функцию Masked language modelling, требуется реализовать вычисление десяти самых вероятных слов, на месте любого умышленно пропущенного слова в корректно составленном предложении на русском языке.

Каждому студенту преподавателем будет дана пара слов, и требуется построить окружение, т. е. само возможное предложение на русском языке с пропущенным словом, для которого в вариантах подстановки пара слов будет встречаться в первой десятке. Слова должны совпадать с точностью до словоформы (слово «домами» не может подходить под требуемое слово «домом»).

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Загружаем предобученную модель BERT для русского языка
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.

In [ ]:
def predict_masked_word(sentence, target_word):
    # Токенизируем предложение
    input_ids = tokenizer.encode(sentence, return_tensors='pt')

    # Находим индекс маскированного слова
    mask_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    # Делаем предсказание
    with torch.no_grad():
        logits = model(input_ids)[0]

    # Получаем вероятности для каждого токена
        masked_index_logits = logits[0, mask_index]
        top_k_probs, top_k_indices = torch.topk(masked_index_logits, 10)

    # Декодируем индексы обратно в слова
        predicted_tokens = tokenizer.convert_ids_to_tokens(top_k_indices)

    return predicted_tokens

In [ ]:
target_words = ["будет", "идёт"]

# Примеры предложений (замените их на свои)
sentences = [
    "Завтра я на работу, а после работы я ... домой.",
    "Погода сегодня хорошая, можно ... гулять.",
    # ... другие предложения
]

for sentence in sentences:
    for word in target_words:
        masked_sentence = sentence.replace(word, "[MASK]")
        predictions = predict_masked_word(masked_sentence, word)
        print(f"Sentence: {sentence}")
        print(f"Target word: {word}")
        print(f"Predictions: {predictions}")
        print()

Sentence: Завтра я на работу, а после работы я ... домой.
Target word: будет
Predictions: []

Sentence: Завтра я на работу, а после работы я ... домой.
Target word: идёт
Predictions: []

Sentence: Погода сегодня хорошая, можно ... гулять.
Target word: будет
Predictions: []

Sentence: Погода сегодня хорошая, можно ... гулять.
Target word: идёт
Predictions: []

